In [1]:
%matplotlib inline

from time import time

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np


TIMESTAMP = int(time())

## Dados

Carregamos os dados já embaralhados divididos em train e test

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Passamos as entradas pra `float` (pra poder manipular), adicionamos a dimensão do canal e pré-tratamos.

Como temos um intervalo definido de entrada `[0, 255]`, simplesmente mudamos a escala para `[0, 1]`, o que é bem comum de se fazer em imagens.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train /= 255
x_test /= 255

Passamos os labels pra one-hot encoding (vetor 10-dimensional)

In [4]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Modelo

Camada de entrada (compatível com a forma de x)

In [5]:
out = entry = Input(shape=x_train.shape[1:])

*(Opcional)* Camada de convolução

In [ ]:
out = Conv2D(32, kernel_size=3, strides=1)(out)
out = Activation('relu')(out)
out = MaxPool2D()(out)

Transformamos o tensor em um vetor unidimensional. Isso  é necessário para podermos aplicar uma camada densa

In [6]:
out = Flatten()(out)

*(Opcional)* Camada intermediária

In [ ]:
out = Dense(20)(out)
out = Activation('relu')(out)

Camada de saída com 10 neurônios, cada um responsável por um dígito e aplicação do softmax para obtermos uma distribuição de probabilidade

In [7]:
out = Dense(10)(out)
out = Activation('softmax')(out)

W0926 13:08:11.557226 140180278683456 deprecation.py:506] From /usr/local/google/home/grovina/.local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Definição do modelo em si

In [8]:
net = Model(entry, out)

Imprimimos a descrição do modelo

In [9]:
net.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


## Treinamento

Definição do custo e da otimização
Custo é a cross-entropia entre saída e resposta
Otimização é a descida de gradientes estocástica

In [10]:
net.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy'])

Treinamento em si

In [12]:
net.fit(
    x_train, y_train,
    batch_size=60,
    epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[
        ModelCheckpoint('save/mnist.{epoch:02d}.h5'),
        TensorBoard(log_dir='logs/mnist_{}'.format(TIMESTAMP))])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3348 - acc: 0.9061 - val_loss: 0.3046 - val_acc: 0.9149
Epoch 2/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3125 - acc: 0.9122 - val_loss: 0.2951 - val_acc: 0.9207
Epoch 3/20
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3010 - acc: 0.9161 - val_loss: 0.2881 - val_acc: 0.9188
Epoch 4/20
60000/60000 [==============================] - 1s 22us/sample - loss: 0.2933 - acc: 0.9180 - val_loss: 0.2825 - val_acc: 0.9205
Epoch 5/20
60000/60000 [==============================] - 2s 26us/sample - loss: 0.2878 - acc: 0.9200 - val_loss: 0.2808 - val_acc: 0.9213
Epoch 6/20
60000/60000 [==============================] - 1s 24us/sample - loss: 0.2838 - acc: 0.9200 - val_loss: 0.2768 - val_acc: 0.9221
Epoch 7/20
60000/60000 [==============================] - 2s 25us/sample - loss: 0.2798 - acc: 0.9218 - val_loss: 0.

## Inferência

Carrega a imagem, converte para escala de cinza e redimensiona para o tamanho da rede

In [ ]:
image = load_img('data/5.png', color_mode='grayscale', target_size=net.input_shape[1:])
answer = to_categorical(5, 10)

In [ ]:
i = np.random.choice(len(x_test))
image, answer = x_test[i], y_test[i]

Cria a entrada e infere a saída

In [ ]:
x = np.reshape(image, (1, 28, 28, 1))
y = net.predict(x, verbose=0)[0]

plt.imshow(x[0, ..., 0], cmap='gray')
plt.show()
print('A rede diz:', np.argmax(y))
print('Gabarito:', np.argmax(answer))